In [1]:
import os
import nltk
import numpy as np
import pandas as pd
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, load_metric, Dataset, DatasetDict
os.environ['CUDA_VISIBLE_DEVICES'] = '1,3'
data_dir="data"
path = data_dir+"/bionlp-workshop-2023-shared-task-1a-problem-list-summarization-1.0.0/BioNLP2023-1A-Train.csv"
df = pd.read_csv(path)
df.head()

/opt/conda/envs/py37/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,File ID,Assessment,Summary,Subjective Sections,Objective Sections
0,188026.txt,"H/O HYPERKALEMIA (HIGH POTASSIUM, HYPERPOTASSE...",# Hypoxia:; Hyperkalemia,ULTRASOUND - At [**2121-3-16**] 11:32 AM\n- MD...,Last dose of Antibiotics:\nAzithromycin - [**2...
1,101616.txt,67 y/o M CAD s/[**Initials (NamePattern4) **] ...,Lower GI bleed; Hypotension; CAD,COLONOSCOPY - At [**2171-1-25**] 12:31 PM\n- C...,Last dose of Antibiotics:\nInfusions:\nOther I...
2,102486.txt,81F with h/o chronic eosinophilic lung disease...,PULMONARY EMBOLISM; FEVER; HYPOTENSION; OLIGUR...,Pleuritic right chest pain\n- patient started ...,Last dose of Antibiotics:\nCiprofloxacin - [**...
3,198989.txt,"79 yo F w/ a h/o CHF (EF of 20-30%), carotid s...",Sepsis; Altered/Depressed MS\n thought to be [...,- ID: rec bedside echo\n- continued fluid bolu...,Last dose of Antibiotics:\nMetronidazole - [**...
4,193604.txt,Mr. [**Known firstname 1908**] [**Known lastna...,# Bradycardia / Rhythm; #. Hypertension; # CAD...,High-grade AV nodal block\n- Had successful [*...,Last dose of Antibiotics:\nInfusions:\nOther I...


In [2]:
for col_name in df.columns:
    print(col_name)
    print(df.iloc[0][col_name])
    print("====================================")

File ID
188026.txt
Assessment
H/O HYPERKALEMIA (HIGH POTASSIUM, HYPERPOTASSEMIA)
 .H/O HYPERGLYCEMIA
 CHRONIC OBSTRUCTIVE PULMONARY DISEASE (COPD, BRONCHITIS, EMPHYSEMA)
   WITH ACUTE EXACERBATION
   A 59 year-old man presents with malaise and hypoxia
Summary
# Hypoxia:; Hyperkalemia
Subjective Sections
ULTRASOUND - At [**2121-3-16**] 11:32 AM
- MD [**First Name (Titles) 2745**] [**Last Name (Titles) 11871**] [**Name (NI) 1270**] but coulnd't leave voicemail because her
voicemail wasn't set up
- patient refused NGT for kayexalate; K 6.5 at noon; ECG unremarkable.
Was able to take PO well in p.m., received kayexalate; K down to 5.0
- renal u/s: no hydronephrosis
No Known Drug Allergies
Changes to  and
f
Review of systems is unchanged from admission except as noted below
Review of systems:
Objective Sections
Last dose of Antibiotics:
Azithromycin - [**2121-3-16**] 10:46 AM
Infusions:
Other ICU medications:
Furosemide (Lasix) - [**2121-3-16**] 02:45 PM
Heparin Sodium (Prophylaxis) - [**21

In [3]:
df['source_text'] = "summarize: " + " <ASSESSMENT> "+ df['Assessment'] + " <SUBJECTIVE> "+ df['Subjective Sections'] +" <OBJECTIVE> " + df['Objective Sections']
df['target_text'] = df["Summary"]

In [4]:
df = df.applymap(str)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=2023)
train_df.shape, test_df.shape

((612, 7), (153, 7))

In [5]:

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
my_dataset_dict = DatasetDict({"train":train_dataset,"test":test_dataset,'validation':test_dataset})
metric = load_metric("rouge")

/opt/conda/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  after removing the cwd from sys.path.


In [6]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = ["hello there", "general kenobi"]
metric.compute(predictions=fake_preds, references=fake_labels)

{'rouge1': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rouge2': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeL': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)),
 'rougeLsum': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0))}

In [7]:

model_checkpoint = "t5-base"
# model_checkpoint = "t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
# if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
#     prefix = "summarize: "
# else:
#     prefix = ""
prefix = ""

In [8]:
max_input_length = 512
max_target_length = 64

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["source_text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["target_text"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [9]:
preprocess_function(my_dataset_dict['train'][:2])

{'input_ids': [[21603, 10, 3, 2, 3291, 134, 10087, 11810, 3155, 6897, 3, 63, 32, 377, 6, 3150, 1695, 113, 6621, 28, 12085, 11, 5738, 1406, 6, 10246, 12, 8, 27, 5211, 21, 10950, 13177, 5, 3, 2, 4138, 279, 683, 14196, 8087, 3155, 272, 5017, 7039, 205, 4254, 25380, 308, 3, 18, 486, 784, 19844, 2658, 2079, 3486, 18, 2555, 19844, 908, 11484, 10, 4118, 3246, 3, 5905, 9730, 205, 4254, 25380, 3, 18, 486, 784, 19844, 2658, 2079, 3486, 18, 2555, 19844, 908, 11484, 10, 4118, 3246, 15971, 434, 12222, 5359, 3, 18, 486, 784, 19844, 2658, 2079, 3486, 18, 2555, 19844, 908, 11484, 10, 4118, 3246, 262, 18256, 3, 18, 486, 784, 19844, 2658, 2079, 3486, 18, 2555, 19844, 908, 14146, 10, 1206, 3246, 784, 19844, 2658, 2079, 3486, 18, 2555, 19844, 908, 10, 17656, 4227, 3, 12380, 19, 4863, 117, 10118, 262, 18256, 24, 3217, 22726, 12110, 1707, 13, 584, 8525, 2904, 26164, 5, 465, 984, 8521, 5, 3, 195, 49, 4044, 7, 10, 4511, 31027, 7, 8475, 5398, 180, 2091, 53, 13456, 17, 5397, 41, 7395, 138, 61, 41, 134, 83, 89, 

In [10]:
tokenized_datasets = my_dataset_dict.map(preprocess_function, batched=True)

100%|██████████| 1/1 [00:00<00:00, 12.36ba/s]


In [11]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [12]:
batch_size = 16//len(os.environ['CUDA_VISIBLE_DEVICES'].split(","))
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"models/{model_name}-finetuned-xsum",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=20,
    predict_with_generate=True,
    fp16=False,
    push_to_hub=False,
)

In [13]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [14]:


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [15]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [16]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: Objective Sections, target_text, source_text, Summary, Assessment, __index_level_0__, File ID, Subjective Sections.
/opt/conda/envs/py37/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 612
  Num Epochs = 20
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 780
/opt/conda/envs/py37/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; wil

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,2.966501,15.178700,3.994000,13.430300,13.431900,18.150300


The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: Objective Sections, target_text, source_text, Summary, Assessment, __index_level_0__, File ID, Subjective Sections.
***** Running Evaluation *****
  Num examples = 153
  Batch size = 16


KeyboardInterrupt: 